In [89]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import ast

In [90]:
df = pd.read_csv("OyoUKFinalwithCity.csv")

In [91]:
df.head

<bound method NDFrame.head of                           single-href  \
0     https://www.oyorooms.com/87452/   
1    https://www.oyorooms.com/210121/   
2     https://www.oyorooms.com/74854/   
3     https://www.oyorooms.com/88080/   
4      https://www.oyorooms.com/2498/   
..                                ...   
371  https://www.oyorooms.com/170754/   
372  https://www.oyorooms.com/211999/   
373  https://www.oyorooms.com/205747/   
374  https://www.oyorooms.com/215369/   
375  https://www.oyorooms.com/210680/   

                                             Title      Price  Dicount  \
0                   SPOT ON The Greenwoods Retreat   ₹ 454.00  76% off   
1               OYO Flagship Hotel Vishal Paradise   ₹ 499.00  77% off   
2                            SPOT ON Kailash Hotel   ₹ 446.00  75% off   
3                          SPOT ON Sneha Home Stay   ₹ 508.00  76% off   
4    Super OYO Collection O Hotel Central Park Inn   ₹ 672.00  76% off   
..                                             ...        ...      ...   
371                        Capital O Tourist Homes   ₹ 759.00  86% off   
372             OYO Flagship The Green Park Shimla   ₹ 764.00  85% off   
373                         Flagship Hotel Apasara  ₹ 1137.00  74% off   
374                   OYO Flagship Kanwar Homestay  ₹ 1492.00  75% off   
375                OYO HOME 81250D Abhiaak Regency  ₹ 1689.00  78% off   

                                           Description  \
0    Did you know that we’ve got 2.5 Crore bookings...   
1                                                  NaN   
2                                                  NaN   
3    Did you know that we’ve got 2.5 Crore bookings...   
4    Did you know that we’ve got 2.5 Crore bookings...   
..                                                 ...   
371  Capital O 86079 Tourist homes is a high-end ho...   
372                                                NaN   
373                                                NaN   
374                                                NaN   
375                                                NaN   

                                                  Html  Rating      City  \
0    <div class="c-riklip striked" itemprop="amenit...     4.7  Dehradun   
1    <div class="c-riklip" itemprop="amenityFeature...     3.4  Dehradun   
2    <div class="c-riklip" itemprop="amenityFeature...     5.0  Dehradun   
3    <div class="c-riklip striked" itemprop="amenit...     3.5  Dehradun   
4    <div class="c-riklip striked" itemprop="amenit...     3.8  Dehradun   
..                                                 ...     ...       ...   
371  <div class="c-riklip striked" itemprop="amenit...     2.7    Shimla   
372  <div class="c-riklip" itemprop="amenityFeature...     3.0    Shimla   
373  <div class="c-riklip" itemprop="amenityFeature...     3.4    Shimla   
374  <div class="c-riklip" itemprop="amenityFeature...     NaN    Shimla   
375  <div class="c-riklip striked" itemprop="amenit...     4.5    Shimla   

                                             Amenities  
0    AC, TV, Free Parking, Daily House Keeping, Att...  
1               AC, Free Wifi, TV, daily House Keeping  
2                                    Free Wifi, AC, TV  
3                                    AC, TV, Free WIfi  
4                                    TV, AC, Free Wifi  
..                                                 ...  
371                                                NaN  
372                                                NaN  
373                                                NaN  
374                                                NaN  
375                                                NaN  

[376 rows x 9 columns]>

In [92]:
df.dtypes

single-href     object
Title           object
Price           object
Dicount         object
Description     object
Html            object
Rating         float64
City            object
Amenities       object
dtype: object

In [93]:
print(df.columns)

Index(['single-href', 'Title', 'Price', 'Dicount', 'Description', 'Html',
       'Rating', 'City', 'Amenities'],
      dtype='object')


In [94]:
df['tags'] = df['Title'] + ' ' + df['Price'] + ' ' + df['Dicount']+ ' ' + df['City'] + ' ' + df['Amenities']

In [95]:
# Testing Phase
# Sample purchase history data (item, color)
# Hotel Name and Location
# Contenst Based Filtering
purchase_history = [
    ("OYO Flagship Hotel Queens Pearl Inn", "Dehradun"),
    ("OYO Hotel Palazzo", "Mussoorie"),
    ("OYO Flagship Hotel Queens Pearl Inn", "Dehradun"),
    ("OYO Home Naini Homes", "Nanital"),
    ("OYO Flagship Hotel Kashi", "Haridwar"),
    ("OYO Flagship Hotel Kashi", "Haridwar"),
    ("OYO Ganga View (Harry Stay)", "Rishikesh"),
    ("Collection O Garden Reach", "Kempty Falls"),
    ("OYO Home Naini Homes", "Nanital"),
    # ... other purchase history items
]
purchase_keywords = [item.lower() for item, _ in purchase_history]

In [96]:
# Testing Phase
def matches_keywords(title):
    lower_title = title.lower()
    return any(keyword in lower_title for keyword in purchase_keywords)

In [97]:
# Filter products based on matching keywords
filtered_products = df[df['Title'].apply(matches_keywords)]


In [98]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet


In [99]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [100]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [101]:
def preprocess(text):
    text = str(text)
    tokens = word_tokenize(text.lower())
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]
    return ' '.join(stemmed_tokens)

In [105]:
def func(user_input):
  c=1
  df['new_tags'] = df['tags'].apply(preprocess)
  preprocessed_input = preprocess(user_input)
  keywords = preprocessed_input.split()
  df['Relevance'] = df['new_tags'].apply(lambda text: sum(keyword in text for keyword in keywords))
  filtered = df[df['Relevance'] > 0]

  flag=0
  sorted_recommendations = filtered.sort_values(by='Relevance', ascending=False)
  top_5_recommendations = sorted_recommendations.head(5)
  if not top_5_recommendations.empty:
      print("Top 5 recommendations are\n")
      for index, row in top_5_recommendations.iterrows():
          print(f"Title: {row['tags']}")
          if(flag==0):
            flag=1
            prompt=row['tags']
          print(f"Link: {row['single-href']}")
        #  print(f"Discount: {row['Discount']}")
          print()
  else:
      print("\nNo matching listings found.")
  print("\nBased on your past purchases!!")
  for i, row in filtered_products.iterrows():
    if(c<=5):
      print()
      print(f"Title: {row['Title']}")
      print(f"Link: {row['single-href']}")
     # print(f"Discount: {row['Discount']}")
      c=c+1
    else :
      break


  # prompt=prompt.replace('(', '')
  # prompt=prompt.replace(')', '')

In [109]:
user_input = input("\nEnter your preference: ")
if "day out", "" not in user_input:
  print("Our Options are\n")
  func(user_input)
else:
  pass



Enter your preference: plan me a day out in delhi for 
